In [28]:
import numpy as np
import pandas as pd

In [29]:
matches = pd.read_csv('data/matches.csv', dtype=str)
treatment_schools = matches[matches['trt'] == '1']['school_code'].tolist()
control_schools = matches[(matches['trt'] != '1') & (~matches['pairmatch'].isna())]['school_code'].tolist()
remnant_schools = matches[matches['pairmatch'].isna()]['school_code'].tolist()
print(len(matches), len(treatment_schools) + len(control_schools) + len(remnant_schools))

521 521


In [23]:
mass_doe_data = pd.read_csv('mcas_exports/csv/mass_doe_data.csv', dtype=str).astype(str)
mass_doe_data = mass_doe_data.set_index('school_code')
float_columns = [c for c in mass_doe_data if c != 'prior_performance']
mass_doe_data[float_columns] = mass_doe_data[float_columns].astype(float)
mass_doe_data['prior_performance'] = mass_doe_data['prior_performance'].apply(lambda x: np.array(eval(x.replace('nan', 'np.nan'))))
mass_doe_data = mass_doe_data[mass_doe_data.index.isin(remnant_schools)]
mass_doe_data = mass_doe_data.sort_values('school_code')

521

In [25]:
remnant_schools

['00010015',
 '00010405',
 '00020010',
 '00020015',
 '00020020',
 '00020025',
 '00030305',
 '00050303',
 '00070013',
 '00080009',
 '00080020',
 '00080050',
 '00090305',
 '00090310',
 '00090350',
 '00100410',
 '00160305',
 '00160315',
 '00160320',
 '00180010',
 '00180510',
 '00200315',
 '00200505',
 '00230305',
 '00240006',
 '00240025',
 '00260305',
 '00280005',
 '00310305',
 '00350004',
 '00350012',
 '00350013',
 '00350020',
 '00350074',
 '00350080',
 '00350096',
 '00350116',
 '00350179',
 '00350182',
 '00350183',
 '00350190',
 '00350229',
 '00350240',
 '00350255',
 '00350257',
 '00350258',
 '00350262',
 '00350308',
 '00350326',
 '00350346',
 '00350363',
 '00350375',
 '00350376',
 '00350377',
 '00350378',
 '00350380',
 '00350382',
 '00350383',
 '00350426',
 '00350445',
 '00350470',
 '00350485',
 '00350545',
 '00350565',
 '00350656',
 '00350657',
 '00350691',
 '00350750',
 '00360325',
 '00370005',
 '00380013',
 '00390005',
 '00400305',
 '00400310',
 '00430005',
 '00440023',
 '00440078',